In [1]:
import pandas as pd
import numpy as np
import time
import funcoes
from datetime import datetime, timedelta
from dateutil import parser

# Configs
pd.set_option("display.date_dayfirst", True)
# Variaveis globais
tks = ['HihrPwM6iplpMc20YafyI0X20hYBwMJjn9O5b3uRXLc%3D','NHvW6AeyLM3HFKp1HwXQIbDbGVdDOyMhvbyHgEoZY57S4pEFqdDDKy1R%2FVQHplHm','UVzI3W1fv8mAK28sWaTc%2BOlAefhF1q26WWTfaOmb%2BSY6F1b2IE43cgRMrafYMySh','oAlfna7l2QNiS6nZo4oIQSQCpNo5XX9iydYqEGfMVkDVz8fBG2Vl6bW9lChTtGtP', 'vcMSbDWqyMNu8fcVLol8l0dZ0DZhinIFKyydHCBSAb%2B5RloWjteXNY9345MIcbUQ']
metodo = 'GET'
url = "https://elysia.zeev.it"
tipoUrl = "/api/2/assignments"
filtro = {'flowId': 0}

# Datas
today = datetime.today().strftime('%Y-%m-%d')
periodo1 = datetime.today() - timedelta(days=7)
periodo1 = periodo1.strftime('%Y-%m-%d')
periodo2 = today 
print("No período de {0} e {1}".format(str(periodo1), str(periodo2)))

No período de 2022-10-31 e 2022-11-07


In [7]:
# Se a API alterar seu campos novamente, sera necessario alterar apenas alguns filtros desta funcao
def request_Orquestra(method='GET',urlReq="https://elysia.zeev.it",typeReq="/api/2/assignments",tokens={},filters={}) -> list:
        try:
                # Variaveis de controle
                row = 1
                i = 1
                retorno = []
                # filtros base para o payload
                if not typeReq.lower().__contains__('report'):
                        filters['recordsPerPage'] = 100
                        seconds = 1
                else: 
                        print("---- >> Requisicao lenta... << ---- ")
                        filters['recordsPerPage'] = 30
                        seconds = 3
                
                filters['mobileEnabledOnly'] = False
                # Faz a busca para cada token
                for tk in tokens:
                        # Reseta variaveis de controle
                        row = 1
                        i = 1
                        # Iteracao com token
                        authorization= {'Authorization': "Bearer " + tk}
                        while row != 0:
                                # Aguardo x segundos devido ao limite de requisicoes - lembrando que a limite de requisicoes por segundos, minutos, horas, dias e meses
                                time.sleep(seconds) 
                                
                                # Incremento de pagina
                                filters['pageNumber'] = i
                                
                                # Requisicao
                                getReq = funcoes.requests_API_Orquestra(metodo=method, 
                                                                        urlAcesso=urlReq, 
                                                                        tipoAcesso=typeReq, 
                                                                        head=authorization, 
                                                                        payload=filters)
                                
                                # Resultado JSON da requisicao
                                result = getReq.json()
                                        
                                # Checa se esta vazio o resultado, o que significa que nao tem mais paginas
                                if (len(result) == 0) or (getReq.status_code != 200):
                                        break
                                else:
                                        # coloca cada retorno na lista
                                        retorno.append(result)
                                        i+=1
                # Cada usuario do token fica em uma sublista da lista de retorno, por isto a necessidade de 'explodir' ela dentro de outra lista.
                retornoLista = [item for sublista in retorno for item in sublista]
                return retornoLista 
        except Exception as e:
                funcoes.printError(e)
                return [] 

In [3]:
def formata_DataFrame(lista:list) -> pd.DataFrame:
    '''
    Funcao
    ------
        Faz a formatacao do data frame colocando as no formato correto e criando uma coluna chamada `atrasada`.
    '''
    # verifica se houve retorno
    try:
        # ocorreu erro ou nao ocorreram retornos - provisorio
        if len(lista) <= 0:
            df = pd.DataFrame
            return df
        # houve retorno de linhas
        elif len(lista) > 0:
            df = pd.DataFrame(lista)
            # Colocando o idTask para conferencia
            df['idTask'] = pd.DataFrame(df['instance'].apply(pd.Series))['id']
            # selecionando apenas as colunas necessarias
            df = df[['idTask','id','taskName','requestName','startDateTime','expirationDateTime','flow','instance']]

            # convertendo colunas para datetime
            df['expirationDateTime'] = pd.to_datetime(df['expirationDateTime'])
            df['startDateTime'] = pd.to_datetime(df['startDateTime'])

            # formatando data em DD/MM/YY
            df.loc[:,'expirationDateTime'] = df.loc[:,'expirationDateTime'].dt.strftime('%Y-%m-%d')
            df.loc[:,'startDateTime'] = df.loc[:,'startDateTime'].dt.strftime('%Y-%m-%d')

            # convertendo colunas para datetime
            df['expirationDateTime'] = pd.to_datetime(df['expirationDateTime'])
            df['startDateTime'] = pd.to_datetime(df['startDateTime'])

            # adiciona coluna de tarefa atrasada
            df['taskAtrasada'] = False
            df.loc[today > df['expirationDateTime'],['taskAtrasada']] = True
            
            return df
    except Exception as e:
        funcoes.printError(e)
        df = pd.DataFrame()
        return df

In [4]:
def contagem_Instancias_Orquestra(nomeSolicitacao:str, nomeTarefa:str,per1:datetime,per2:datetime,dataFrame:pd.DataFrame, status:str='endDateTime') -> int:
    '''
     Funcao
     ------
            Retorna a contagem de tarefas fechadas para determinada solicitacao.

    Parameters
    ----------
        nomeSolicitacao: str (obrigatorio)
            Nome da solicitacao que sera buscada e contada a instancia.
        
        nomeTarefa: str (obrigatorio)
            Nome da tarefa para ser contada a sua quantidade.

        dataFrame: dataFrame Pandas (obrigatorio)
            Dataframe que sera utilizado como base de dados para a contagem.
        
        per1: datetime (obrigatorio)
            Periodo inicial para busca.

        per2: datetime (obrigatorio)
            Periodo final para busca.

        status: str (opcional)
            Status da data para verificar o periodo - podendo ser `endDateTime` ou `startDateTime`.
    Retorno
    -------
        int: Retorna a quantidade de instancias ja finalizadas desta tarefa.
        * Se erro, retorna -1   
    '''
    try:
        # Formatar parametros - remove espacos em branco, deixar em letra minuscula
        nomeSolicitacao = str(nomeSolicitacao).lower().replace(" ","")
        nomeTarefa = str(nomeTarefa).lower()
        # Projetos 
        projetos = dataFrame.loc[(dataFrame['requestName'].str.lower().str.replace(" ","").str.contains(nomeSolicitacao))]
        # Contagem
        cont=0
        # Percorre linha a linha do dataframe, verificando cada instancia aberta ou fechada da solicitacao 
        for row in projetos.values:
            for instance in row[19]:
                task = str(instance['task']['name']) 
                date = parser.parse(instance[status]).strftime("%Y-%m-%d")
                if task.lower().__contains__(nomeTarefa):
                    if date >= per1 and date <= per2:
                        cont+=1
                        break
        return cont
    except Exception as e:
        print('--> Erro na funcao \'cont_Instancias_Orquestra\' <--')
        funcoes.printError(e)
        return -1

def instances_Report_Orquestra(userToken:str, showFinishedTasks:bool=True):
    '''
        Funcao
        ------
            Lista todas instâncias de solicitações que a pessoa relacionada ao token possui permissão de consultar de acordo com filtros.
        
        Parametros
        ----------
            userToken: str (obrigatorio)
                Token do usuario orquestra para solicitar a requisicao.

            showFinishedTasks: boolean (opcional)
                definir se vai filtrar por instancias fechadas ou instancias abertas das solicitacoes.
        Retorno 
        -------
            Retorna um DataFrame(Pandas)
    '''
    try:
        # OBRIGATORIO o filtro de startDateInterval
        f = {"startDateIntervalBegin": "2000-01-01T00:00:00",
                "startDateIntervalEnd": "2030-12-31T23:59:59",
                "showFinishedInstanceTasks": True, 
                "showPendingInstanceTasks": False,
                "active": True}

        # Escolhe se vai mostrar tarefas fechadas ou pendentes
        if showFinishedTasks == False:
            f['showFinishedInstanceTasks'] = False
            f['showPendingInstanceTasks'] = True

        # Requisicao 
        lista = request_Orquestra(method='GET',
                                    urlReq=url,
                                    typeReq='/api/2/instances/report',
                                    tokens=[userToken],
                                    filters=f)
        df = pd.DataFrame(lista)
        return df
    except Exception as e:
        print('--> Erro na funcao \'instances_Report_Orquestra\' <--')
        funcoes.printError(e)
        df = pd.DataFrame()
        return df

In [72]:
def contagem_CS1(dataFrame:pd.DataFrame) -> list:
    '''
        Funcao
        ------
            Faz a contagem de clientes no fluxo CS1.
    '''
    try:
        # CS1 Abertos sao os Efetivos Em espera + Em Aberto nas atribuicoes de cada usuario -> Pegar o relogio de 7/15 dias
        df = dataFrame
        cs1 = df.loc[(df['requestName'].str.lower().str.replace(" ","").str.contains('projetov.20')) & ((df['taskName'].str.lower().str.contains('ligação do cs')) | (df['taskName'].str.lower().str.replace(" ","").str.contains('cooldown')))]
        contCs1Abertos = len(cs1)

        # CS1 Efetivos sao os Em espera do 'Servico Orquestra'
        # Entrou no relogio de 7 ou 15 dias na semana do script, conta como efetivo
        cs1Efetivos = df.loc[(df['requestName'].str.lower().str.replace(" ","").str.contains('projetov.20')) 
                            & (df['taskName'].str.lower().str.replace(" ","").str.contains('cooldown'))
                            & (df['startDateTime'].between(periodo1,periodo2, inclusive='both'))]
        contCs1Efetivos = len(cs1Efetivos)

        # CS1 Atrasados 
        contCS1Atrasadas = len(cs1.loc[(df['taskAtrasada'] == True) & (df['taskName'].str.lower().str.contains('ligação do cs'))])
        return [contCs1Abertos,contCs1Efetivos,contCS1Atrasadas]
    except Exception as e:
        print('--> Erro na funcao \'contagem_CS1\' <--')
        funcoes.printError(e)
        return []

def contagem_CS2_Degustacao(instancesDataFrame:pd.DataFrame) -> int:
    try:
        # Formatar parametros - remove espacos em branco, deixar em letra minuscula
        nomeSolicitacao = str('degustação').lower().replace(" ","")
        nomeTarefa = str('ligação do cs').lower()
        # Projetos 
        projetos = instancesDataFrame.loc[(instancesDataFrame['requestName'].str.lower().str.replace(" ","").str.contains(nomeSolicitacao))]
        # Contagem
        cont=0
        aberto=0
        # Percorre linha a linha do dataframe, verificando cada instancia aberta ou fechada da solicitacao 
        for row in projetos.values:
            for instance in row[19]:
                task = str(instance['task']['name']) 
                if task.lower().__contains__(nomeTarefa):
                    cont+=1
            if cont>=1:
                aberto+=1 # conta a quantidade de abertos em degustacao
                cont=0
        return aberto
    except Exception as e:
        funcoes.printError(e)
        return -1

def contagem_CS2(dataFrame:pd.DataFrame, instancesDataFrame:pd.DataFrame) -> list:
    '''
        Funcao
        ------
            Faz a contagem de clientes no fluxo CS2.
    '''
    try:
        df = dataFrame
        # CS2 Em aberto -> Para degustacao a etapa 'Inicio do mes seguinte' contabiliza os em aberto apos email = acomp personalizado 1/3
        cs2 = df.loc[(df['requestName'].str.lower().str.contains('degustação')) | (df['requestName'].str.lower().str.contains('normal')) & ~(df['taskName'].str.lower().str.contains('relatório')) | (df['taskName'].str.lower().str.contains('suporte'))]
        cs2Degustacao = contagem_CS2_Degustacao(instancesDataFrame)
        cs2Normal = contagem_fluxo_personalizado(instancesDataFrame)
        contCs2Abertos =  cs2Degustacao + cs2Normal
        # CS2 Efetivos -> Necessario contabilizar apenas a quarta ligacao como efetivo
        cs2Efetivo = cs2.loc[~(cs2['taskName'].str.lower().str.contains('ligação do cs'))]
        contCs2Efetivos = len(cs2Efetivo)
        # CS2 Atrasados
        cs2Atrasados = cs2.loc[(cs2['taskName'].str.lower().str.contains('ligação do cs')) & (cs2['taskAtrasada'] == True)]
        contCs2Atrasados = len(cs2Atrasados)
        return [contCs2Abertos,contCs2Efetivos,contCs2Atrasados]
    except Exception as e:
        funcoes.printError(e)
        return []

def contagem_fluxo_personalizado(instancesDataFrame:pd.DataFrame) -> int:
    '''
        Funcao
        ------
           Faz a contagem de clientes no fluxo personalizado.
    '''
    try:
        df = instancesDataFrame
        contClientesfluxoNormal = len(df.loc[df['requestName'].str.lower().str.contains("normal")]) 
         # Retorno com a contagem
        return contClientesfluxoNormal
    except Exception as e:
        print('--> Erro na funcao \'fluxo_personalizado\' <--')
        funcoes.printError(e)
        return -1

def contagem_layouts_documentos(dataFrame:pd.DataFrame,instancesDataFrame:pd.DataFrame) -> list:
    try:
        df = dataFrame
        # Envio de Layout Pendentes 
        envLayout = df.loc[(df['requestName'] == 'Projeto  v. 20') & (df['taskName'] == 'Baixar Layout e apresentar ao cliente')]
        contEnvLayoutPendentes = len(envLayout)
        # Envio de Layout Realizados - Contagem por Períodos
        contEnvLayoutRealizados = contagem_Instancias_Orquestra('projeto v.20', 'baixar layout', periodo1, periodo2, instancesDataFrame)
        # Envio de Layout Atrasados
        contEnvLayoutAtrasadas = len(envLayout[(envLayout['taskName'] == 'Baixar Layout e apresentar ao cliente') & (envLayout['taskAtrasada'] == True)])

        # Envio de Documentos Pendentes
        envDoc = df.loc[(df['requestName'] == 'Projeto  v. 20') & (df['taskName'] == 'Entregar Documentação ao Cliente')]
        contEnvDocPendentes = len(envDoc)
        # Envio de Documentos Realizados - Contagem por Períodos
        contEnvDocRealizados = contagem_Instancias_Orquestra('projeto v.20','entregar documentação', periodo1, periodo2, instancesDataFrame)
        # Envio de Documentos Atrasados 
        contEnvDocAtrasados = len(envDoc.loc[envDoc['taskAtrasada'] == True])

        return [[contEnvLayoutPendentes,contEnvLayoutRealizados,contEnvLayoutAtrasadas],[contEnvDocPendentes,contEnvDocRealizados,contEnvDocAtrasados]]
    except Exception as e:
        funcoes.printError(e)
        return []


def main_Func():
    # Requisicao com dados de atribuicoes aos usuarios
    lista = request_Orquestra(method=metodo,urlReq=url,typeReq=tipoUrl,tokens=tks)
    df = formata_DataFrame(lista)

    # Requisicao com dados de instancias 
    instancesReport = instances_Report_Orquestra('enb4iHROSDegvkiZv17Fxw%2BkQ7q7Zidu1PKalcpC4o4%3D')

    contagem_CS1(df)
    contagem_CS2(df)
    contagem_fluxo_personalizado(instancesReport)
    contagem_layouts_documentos(df, instancesReport)

    #print("Contatos Em Aberto CS1 - {0} Contatos Efetivos CS1 - {1} <- No período de {3} e {4} Contatos Em Atraso CS1 - {2}".format(contCs1Abertos,contCs1Efetivos,contCS1Atrasadas,str(periodo1), str(periodo2)))
    #print("Contatos Em Aberto CS2 - {0} Contatos Efetivos CS2 - {1} Contatos Em Atraso CS2 - {2}".format(contCs2Abertos, contCs2Efetivos,contCs2Atrasados))
    #print("Acompanhamento Personalizado - {0} <- No periodo de {1} e {2}".format(contClientesfluxoNormal, str(periodo1), str(periodo2)))
    #print("Envio de Layouts Pendentes  - {0} Envio de Layouts Realizados - {1} <- No periodo de {3} e {4} #Envio de Layouts Atrasados  - {2}".format(contEnvLayoutPendentes, contEnvLayoutRealizados, contEnvLayoutAtrasadas, str(periodo1), str(periodo2)))
    #print("Envio de Documentos Pendentes - {0} Envio de Documentos Realizados  - {1} <- No periodo de {3} e {4} #Envio de Documentos Atrasados - {2}".format(contEnvDocPendentes, contEnvDocRealizados, contEnvDocAtrasados, str(periodo1), str(periodo2)))

In [ ]:
# Requisicao com dados de atribuicoes aos usuarios
lista = request_Orquestra(method=metodo,urlReq=url,typeReq=tipoUrl,tokens=tks)
df = formata_DataFrame(lista)

# Requisicao com dados de instancias 
instancesReport = instances_Report_Orquestra('enb4iHROSDegvkiZv17Fxw%2BkQ7q7Zidu1PKalcpC4o4%3D')

In [74]:
print(contagem_CS1(df))
print(contagem_CS2(df,instancesReport)) 
print(contagem_fluxo_personalizado(instancesReport)) 
print(contagem_layouts_documentos(df, instancesReport))

[67, 37, 19]
[83, 15, 16]
23
[[3, 4, 1], [4, 0, 3]]
